In [1]:
from pathlib import Path
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
import pandas as pd
import numpy as np

data_path = 'path/to/datasets/'
save_path = Path(data_path)

## Decomposition

In [3]:
from statsmodels.tsa.seasonal import STL
from tqdm import trange

def measure_strength(df, dataset, win=0):
    """
    Measures the strength of trend (F_t) and seasonality (F_s) in time series data.

    Parameters:
    - df (pd.DataFrame): The input data containing time series columns.
    - dataset (str): The name of the dataset to identify frequency or specific configurations.
    - win (int): Window size for decomposition; if 0, applies decomposition on the full time series.

    Outputs:
    Prints the average strength of trend and seasonality for the dataset.
    """
    # Decompose the time series for each dimension
    dim_list = ts_decompose(df, dataset, win=win)
    
    F_t_list = []  # List to store trend strength values
    F_s_list = []  # List to store seasonality strength values
    
    for res in dim_list:
        # Skip calculations if variance of the decomposed components is zero
        if (res.trend + res.resid).var() == 0 or (res.seasonal + res.resid).var() == 0:
            continue
        
        # Calculate trend strength (F_t)
        F_t = max(0, 1 - (res.resid.var() / (res.trend + res.resid).var()))
        F_t_list.append(F_t)
        
        # Calculate seasonality strength (F_s)
        F_s = max(0, 1 - (res.resid.var() / (res.seasonal + res.resid).var()))
        F_s_list.append(F_s)
    
    # Print summary of results
    print('dataset: {dataset}, \t win. size: {win},\t Avg. F_t: {avg_ft:2.4f},\t Avg. F_s: {avg_fs:2.4f}'.format(
        dataset=dataset, win=win, avg_ft=np.mean(F_t_list), avg_fs=np.mean(F_s_list)
    ))

def ts_decompose(df, dataset, win=0):
    """
    Decomposes time series data into trend, seasonal, and residual components.

    Parameters:
    - df (pd.DataFrame): The input data containing time series columns.
    - dataset (str): The name of the dataset to identify frequency or specific configurations.
    - win (int): Window size for decomposition; if 0, applies decomposition on the full time series.

    Returns:
    - dim_list (list): A list of decomposition results for each dimension of the time series.
    """
    # Define frequency mapping for datasets
    freq_dict = {
        'ETT-small/ETTh1': 'H', 'ETT-small/ETTh2': 'H', 'ETT-small/ETTm1': 'T', 'ETT-small/ETTm2': 'T',
        'electricity/electricity': 'H', 'exchange_rate/exchange_rate': 'B',
        'illness/national_illness': 'W', 'traffic/traffic': 'H', 'weather/weather': 'T',
        'exchange_rate_nips': 'B', 'solar_nips': 'H', 'electricity_nips': 'H',
        'traffic_nips': 'H', 'wiki2000_nips': 'D'
    }
    
    # Define minimum period mapping for datasets
    min_dict = {
        'ETT-small/ETTm1': (24 * 60) // 15, 'ETT-small/ETTm2': (24 * 60) // 15,
        'weather/weather': (24 * 60) // 10
    }

    dim = len(df.iloc[0])  # Number of dimensions (columns) in the data
    dim_list = []  # List to store decomposition results for each dimension
    
    for i in trange(dim):  # Iterate over each column in the dataset
        if win == 0:
            # Standardize the time series column
            tmp_df = (df.iloc[:, i] - df.iloc[:, i].mean()) / (df.iloc[:, i].std())
            
            # Perform STL decomposition with appropriate frequency settings
            if dataset in freq_dict and freq_dict[dataset] == 'T':
                stl = STL(tmp_df.fillna(0), period=7, robust=True)
            else:
                stl = STL(tmp_df.fillna(0), robust=True)
            
            res = stl.fit()  # Fit the decomposition model
            dim_list.append(res)  # Store the result
        else:
            # Perform windowed decomposition
            right = win  # Initialize the right boundary of the window
            while right < len(df.iloc[1:, i]):
                tmp_df = df.iloc[right - win:right, i]  # Extract the windowed data
                tmp_df = (tmp_df - tmp_df.mean()) / (tmp_df.std())  # Standardize the windowed data
                
                # Perform STL decomposition with appropriate frequency settings
                if dataset in freq_dict and freq_dict[dataset] == 'T':
                    stl = STL(tmp_df.fillna(0), period=7, robust=True)
                else:
                    stl = STL(tmp_df.fillna(0), robust=True)
                
                res = stl.fit()  # Fit the decomposition model
                right += win  # Move the window forward
                dim_list.append(res)  # Store the result
        
    return dim_list  # Return the list of decomposition results

## Normality

In [4]:
from scipy.stats import normaltest
import numpy as np
import scipy.stats
from scipy.stats import norm

def test_normal(df, dataset, win=0):
    dim = len(df.iloc[0])
    score_list = []
    gaussian_count = 0
    count = 0
    for i in range(dim):
        # z-score
        # df.iloc[:,i]=(df.iloc[:,i]-df.iloc[:,i].mean())/(df.iloc[:,i].std())
        value = df.iloc[:,i].dropna().values
        if len(value) < 10:
            continue
        
        right = win
        pvalue = []
        if win > 0:
            while right < len(value):
                res = normaltest(value[right-win:right])[1]
                pvalue.append(res)
                right += win
            res = np.mean(pvalue)
        else:
            res = normaltest(value)[1]
            # res = kstest(value, 'norm')[1]
            if sum(value) == 0:
                continue
            
        if res >= 0.05:
            gaussian_count += 1
        count += 1
            
        score_list.append(res)

    
    print(dataset, " gaussian pvalue: ", str(np.mean(score_list)), '\t gaussian ratio: ', str(gaussian_count/count))


def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M, base=2)+0.5*scipy.stats.entropy(q, M, base=2)

def JS_div(arr1,arr2,num_bins):
    max0 = max(np.max(arr1),np.max(arr2))
    min0 = min(np.min(arr1),np.min(arr2))
    bins = np.linspace(min0-1e-4, max0-1e-4, num=num_bins)
    
    PDF1 = pd.cut(arr1,bins,duplicates='drop').value_counts()
    PDF2 = pd.cut(arr2,bins, duplicates='drop').value_counts()
    
    if sum(PDF1) > 0 and sum(PDF2) > 0:
        PDF1 = PDF1 / len(arr1)
        PDF2 = PDF2 / len(arr2)
        return JS_divergence(PDF1.values,PDF2.values)
    else:
        return None


def cal_JS_divergence(df, dataset, win=0):
    
    dim = len(df.iloc[0])
    js_list = []
    for i in range(1, dim):
        
        # z-score
        global_mu = df.iloc[:,i].mean()
        global_std = df.iloc[:,i].std()
        df.iloc[:,i]=(df.iloc[:,i]-global_mu) / global_std
        value = df.iloc[:,i].dropna().values
        
        if sum(value) == 0:
            continue
        
        right = win
        dim_list = []
        if win > 0:
            while right < len(value):
                tmp_value = value[right-win:right]
                mu = tmp_value.mean()
                std = tmp_value.std()

                norm_dist = norm.rvs(loc=mu, scale=std, size=len(tmp_value))
                res = JS_div(tmp_value,norm_dist,num_bins=20)
                if res is not None:
                    dim_list.append(res)
                right += win
                
            js_div = np.mean(dim_list)

        else:
            norm_dist = norm.rvs(loc=global_mu, scale=global_std, size=len(value))
            js_div = JS_div(value,norm_dist,num_bins=20)
        
        if js_div is not None:
            js_list.append(js_div)
        
    print("window size: ", win, "\t dataset: ", dataset, "\t JS DIV avg: ", str(np.mean(js_list)))

## Long-term Datasets

In [6]:
def load_csv_data(filename, dataset):
    """
    Loads time series data from a CSV file and processes it based on dataset-specific requirements.

    Parameters:
    - filename (str): Path to the directory containing the CSV file.
    - dataset (str): Name of the dataset to be loaded, used for specific handling.

    Returns:
    - df (pd.DataFrame): Processed DataFrame with time series data, indexed by date.
    """
    # Dictionary to map dataset names to their respective data frequency
    freq_dict = {
        'ETT-small/ETTh1': 'H', 'ETT-small/ETTh2': 'H', 'ETT-small/ETTm1': 'T', 'ETT-small/ETTm2': 'T',
        'electricity/electricity': 'H', 'exchange_rate/exchange_rate': 'D',
        'illness/national_illness': 'D', 'traffic/traffic': 'H', 'weather/weather': 'T'
    }

    # Special handling for 'caiso' dataset
    if 'caiso' in dataset:
        # Load the dataset and convert the 'Date' column to datetime
        data = pd.read_csv(filename + dataset + '.csv')
        data['Date'] = data['Date'].astype('datetime64[ns]')
        
        # Names of zones in the dataset
        names = ['PGE', 'SCE', 'SDGE', 'VEA', 'CA ISO', 'PACE', 'PACW', 'NEVP', 'AZPS', 'PSEI']
        
        # Create a DataFrame with a complete hourly date range
        df = pd.DataFrame(pd.date_range('20130101', '20210630', freq='H')[:-1], columns=['Date'])
        
        # Process each zone's data and merge into a single DataFrame
        for name in names:
            current_df = (
                data[data['zone'] == name]
                .drop_duplicates(subset='Date', keep='last')  # Remove duplicate entries, keeping the last
                .rename(columns={'load': name})  # Rename 'load' column to the zone name
                .drop(columns=['zone'])  # Drop the 'zone' column
            )
            df = df.merge(current_df, on='Date', how='outer')  # Merge with the main DataFrame
        
        # Rename the 'Date' column to 'date'
        df = df.rename(columns={'Date': 'date'})
    elif 'nordpool' in dataset:
        # Special handling for 'nordpool' dataset: Parse the 'Time' column as datetime
        df = pd.read_csv(filename + dataset + '.csv', parse_dates=['Time'])
        df = df.rename(columns={'Time': 'date'})  # Rename the 'Time' column to 'date'
    else:
        # General case: Load the dataset as-is
        df = pd.read_csv(filename + dataset + '.csv')
    
    # Convert the 'date' column to datetime format and set it as the index
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')

    # Drop the first column (usually an index column or non-relevant column)
    df = df.iloc[:, 1:]
    
    return df  # Return the processed DataFrame

In [8]:
dataset = 'ETT-small/ETTh1' # 'exchange_rate/exchange_rate'
win_len = 0
df = load_csv_data(data_path, dataset)
measure_strength(df, dataset, win=win_len)

100%|██████████| 6/6 [00:10<00:00,  1.67s/it]

dataset: ETT-small/ETTh1, 	 win. size: 0,	 Avg. F_t: 0.7728,	 Avg. F_s: 0.4772


In [8]:
dataset = 'ETT-small/ETTh1' # 'exchange_rate/exchange_rate'
win_len = 336
df = load_csv_data(data_path, dataset)
cal_JS_divergence(df, dataset, win=win_len)

window size:  336 	 dataset:  ETT-small/ETTh1 	 JS DIV avg:  0.0719988819816385


## Short-term Datasets

In [9]:
def load_prob_data(dataset, win=0):
    freq_dict = {'exchange_rate_nips':'B','solar_nips':'H','electricity_nips':'H','traffic_nips':'H', 'wiki2000_nips':'D'}
    
    idx = 0
    dataname = dataset
    dataset = get_dataset(dataset, path=save_path, regenerate=False)
    dim = int(dataset.metadata.feat_static_cat[0].cardinality)
    train_grouper = MultivariateGrouper(max_target_dim=dim)
    dataset_train = train_grouper(dataset.train)
    data = list(dataset_train)[0]['target']
    start_date = dataset_train[0]['start'].to_timestamp()
    
    # multi
    idx = [i for i in range(dim)]

    data = data.transpose(1,0)
    df = pd.DataFrame(data,columns=idx,dtype=float)

    df['date'] = pd.date_range(start_date,periods=len(data),freq=freq_dict[dataname]) 
    df = df.set_index('date')
        
    return df


In [10]:
# "exchange_rate_nips", "solar_nips", "electricity_nips", "traffic_nips", "taxi_30min", "wiki2000_nips"
dataset = "exchange_rate_nips"
df = load_prob_data(dataset, win=0)

measure_strength(df, dataset, win=0)
cal_JS_divergence(df, dataset, win=30)

/home/v-zhangjiaw/miniconda3/envs/probts/lib/python3.10/site-packages/gluonts/dataset/common.py:263: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.Period(val, freq)
/home/v-zhangjiaw/miniconda3/envs/probts/lib/python3.10/site-packages/gluonts/dataset/multivariate_grouper.py:114: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  timestamp + len(data[FieldName.TARGET]) - 1,
/home/v-zhangjiaw/miniconda3/envs/probts/lib/python3.10/site-packages/gluonts/dataset/multivariate_grouper.py:243: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  index=pd.period_range(
/home/v-zhangjiaw/miniconda3/envs/probts/lib/python3.10/site-packages/gluonts/dataset/multivariate_grouper.py:243: FutureWarning: PeriodDtype[B] is deprecated 

dataset: exchange_rate_nips, 	 win. size: 0,	 Avg. F_t: 0.9982,	 Avg. F_s: 0.1256
window size:  30 	 dataset:  exchange_rate_nips 	 JS DIV avg:  0.2964380648448922
